In [1]:
%matplotlib notebook

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import cv2
import csv

from matplotlib import pyplot as plt

import pandas as pd

## Import previous data

In [2]:
def importRecords():
    a = csv.DictReader(open('data.csv', encoding='utf-8'), delimiter=',', skipinitialspace=True)
    return [x for x in a]

## Website management

In [3]:
def getCountry(name):
    for r in records:
        if r["country"] == name:
#             print(r)
            return [r["Xcentroid"], r["Ycentroid"]]
        
    return False

def saveCountry(name, Xcentroid, Ycentroid):
    records.append({'country': name, 'Xcentroid': Xcentroid, 'Ycentroid':Ycentroid})
    
def saveRecords():
    with open('data.csv', 'w', newline='', encoding='utf-8') as f:
        dict_writer = csv.DictWriter(f, records[0].keys())
        dict_writer.writeheader()
        dict_writer.writerows(records)
        
#print(getCountry("Quentin"))
#print(getCountry("Unkown"))

#saveCountry("Emile", 12, 15)
#print(getCountry("Emile"))
#saveRecords()

In [4]:
def initGame():
    # Initialize the webdriver
    options = Options()
    options.add_extension(r'.\extensions\1.42.4_1.crx')
    options.add_argument("--start-maximized")
    options.add_experimental_option("prefs", { 
        "profile.default_content_setting_values.notifications": 1 
    })
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)

    # Open the websitet
    #browser.get('https://www.jeux-geographiques.com/jeux-en-ligne-Jeu-Pays-d-Afrique-_pageid158.html')
    browser.get('https://www.jeux-geographiques.com/jeux-en-ligne-Jeu-Pays-d-Amerique-du-Sud-_pageid162.html')
    cookies_button = browser.find_element(By.XPATH, "//button[text()=\"J'ACCEPTE\"]").click()
    browser.execute_script("document.getElementById('canvas').scrollIntoView();")
    browser.find_element(By.ID,'buttonStartWithoutTimer').click()
    
    time.sleep(2)
    
    return browser

def closeGame(browser):
    browser.quit()

## Game management

In [5]:
w_page = 1920
h_page = 1080

y_scroll = 1076

x_canva = 310
y_canva = 1076
w_canva = 900
h_canva = 500

x_crop = 0
y_crop = 50
w_crop = w_canva
h_crop = h_canva

def screenshotToSeleniumCoordinates(x_screenshot,y_screenshot) :
    # transposes screenshot xy coordinates (relative to left top corner of img), to selenium xy coordinates (relative to center of element(canvas))
    x_selenium = x_screenshot-448
    y_selenium = y_screenshot-248
    return x_selenium,y_selenium

def findCountryPosition():
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.4)
    canvas.screenshot("screenshot.PNG")

    background = cv2.imread('bg_southamerica.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]
    after = cv2.imread('screenshot.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]

    diff = cv2.absdiff(background, after)
    ret,thresh = cv2.threshold(diff,10,255,cv2.THRESH_BINARY)
    canny_edges = cv2.Canny(thresh, 30, 200)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    cXlist=[]
    cYlist=[]

    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cXlist.append(cX)
            cYlist.append(cY)
        else:
            pass # do nothing, centroid value is not usable
            
    if len(cXlist) > 0:
#         cv2.circle(diff, (cXlist[0], cYlist[0]), 5, (255, 0, 0), -1)
#         cv2.putText(diff, "centroid", (cXlist[0] - 25, cYlist[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#         plt.imshow(diff)
#         plt.show()
        return cXlist[0],cYlist[0]
    else:
        return False # no centroid found in list 
        
        
        
    
def clickOnCanva(x_click, y_click):
    action.move_to_element(canvas).perform()
    action.move_by_offset(x_click,y_click).click().perform()
    
def actionCountry(name):
    print(name)
    country_coordinates = getCountry(name)
    
    if not country_coordinates:
        print("Country not found in records, trying to find it ...")
        coordinates = findCountryPosition()
        
        if not coordinates:
            print("Unable to find the centroid")
            
        else:
            
            x_click,y_click = screenshotToSeleniumCoordinates(coordinates[0],coordinates[1])
            print("Centroid found, adding country to records :")
            print(name, coordinates)
#             print(coordinates[0],coordinates[1])
            saveCountry(name, x_click, y_click)
        
    else:
        print("Country found in records :")
        print(name, country_coordinates)
        clickOnCanva(country_coordinates[0], country_coordinates[1])
    
    print("\n")



## Run

### Learn

In [6]:
records = importRecords()

## Learn & win
for game in ["Learn" , "Win"]:
    print(game)
    
    browser = initGame()
    action = ActionChains(browser)
    canvas = browser.find_element(By.ID,'canvas')
    canvas.screenshot("bg_southamerica.PNG") # save background for later

    for i in range(15):

        time.sleep(2) # wait for country name to appear
        name = browser.find_element(By.ID,'questionTextLabel').text

        actionCountry(name)

    saveRecords()
    
    time.sleep(2)
    closeGame(browser)
    



====== WebDriver manager ======


Learn


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\emile\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\emile\AppData\Local\Temp\ipykernel_10024\2623150995.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)


Le Venezuela
Country not found in records, trying to find it ...
Centroid found, adding country to records :
Le Venezuela (396, 60)


Le Paraguay
Country found in records :
Le Paraguay ['0', '0']


Le Brésil
Country not found in records, trying to find it ...
Centroid found, adding country to records :
Le Brésil (507, 108)


Le Chili
Country not found in records, trying to find it ...
Centroid found, adding country to records :
Le Chili (401, 476)


Les îles Malouines
Country not found in records, trying to find it ...
Centroid found, adding country to records :
Les îles Malouines (439, 456)


L'Argentine
Country not found in records, trying to find it ...
Centroid found, adding country to records :
L'Argentine (407, 471)


La Géorgie du Sud
Country not found in records, trying to find it ...
Centroid found, adding country to records :
La Géorgie du Sud (529, 477)


Le Pérou
Country not found in records, trying to find it ...
Centroid found, adding country to records :
Le Pérou (337, 1



====== WebDriver manager ======


Win


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\emile\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


La Colombie
Country found in records :
La Colombie [-99, -166]


Les îles Malouines
Country found in records :
Les îles Malouines [-9, 208]


La Guyane Française
Country found in records :
La Guyane Française [36, -165]


La Géorgie du Sud
Country found in records :
La Géorgie du Sud [81, 229]


Le Brésil
Country found in records :
Le Brésil [59, -140]


Le Guyana
Country found in records :
Le Guyana [-2, -186]


L'Équateur
Country found in records :
L'Équateur [-213, -118]


Le Chili
Country found in records :
Le Chili [-47, 228]


L'Uruguay
Country found in records :
L'Uruguay [12, 85]


La Bolivie
Country found in records :
La Bolivie [-40, -29]


Le Suriname
Country found in records :
Le Suriname [18, -167]


L'Argentine
Country found in records :
L'Argentine [-41, 223]


Le Paraguay
Country found in records :
Le Paraguay ['0', '0']


Le Pérou
Country found in records :
Le Pérou [-111, -101]


Le Venezuela
Country found in records :
Le Venezuela [-52, -188]




In [7]:
#closeGame(browser)